<a href="https://colab.research.google.com/github/antahiap/dsr-nlp/blob/main/notebooks/06_Finetuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip  uninstall -y transformers tokenizers datasets accelerate evaluate
!pip  install transformers==4.28.0 tokenizers==0.13.3 datasets==2.12.0 accelerate==0.19.0 evaluate==0.4.0

Found existing installation: transformers 4.28.0
Uninstalling transformers-4.28.0:
  Successfully uninstalled transformers-4.28.0
Found existing installation: tokenizers 0.13.3
Uninstalling tokenizers-0.13.3:
  Successfully uninstalled tokenizers-0.13.3
Found existing installation: datasets 2.12.0
Uninstalling datasets-2.12.0:
  Successfully uninstalled datasets-2.12.0
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Found existing installation: evaluate 0.4.0
Uninstalling evaluate-0.4.0:
  Successfully uninstalled evaluate-0.4.0
  Using cached transformers-4.28.0-py3-none-any.whl (7.0 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)


In [2]:
!nvidia-smi

Tue Aug 22 11:36:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import evaluate
import numpy as np
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from datasets import load_dataset, load_metric

# Dataset
IMDB data [link](https://huggingface.co/datasets/imdb)

In [4]:
raw_datasets = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
raw_datasets['test'][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

# Pre trained model
[Hugging phase](https://huggingface.co/bert-base-uncased)

In [7]:
pretrained_model_id = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_id)

In [8]:
tokenizer('Let us check thi token thingi and get to know how it works.')

{'input_ids': [101, 2292, 2149, 4638, 16215, 2072, 19204, 2518, 2072, 1998, 2131, 2000, 2113, 2129, 2009, 2573, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
def tockenize_function(example):
  return tokenizer(example["text"], truncation=True)

In [10]:

tokenized_datasets = raw_datasets.map(tockenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
print(tokenized_datasets['train'][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # knows the tokenizer property

In [13]:
def compute_metrics(eval_preds):
  metric = evaluate.load("accuracy", "f1")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)


  accuracy_metric = load_metric("accuracy")
  f1_metric = load_metric("f1")

  accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
  f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")  # for macro F1

  return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}


# Training

In [14]:
training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy = "epoch",
    per_device_eval_batch_size=24,
    per_device_train_batch_size=24,
    fp16=True,                                 # change the data type from float32 to float16, loose percision and get smaller > half memory
    num_train_epochs=1
)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_id,
    num_labels=2
)
model.cuda()  # runing on gpu
print(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

- in_features=3072: vocab size
- word vec length; 768
- 512: sequence length
- pooling from 512, 768 > 768

[512 * 768] >> Bert Encoder >> [512 * 768] >> Pooling, trainable >> [1 * 768] >> FC, trainable  >> 0/1 labels

In [20]:
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets['train'], #.select(range(100)),
    eval_dataset =tokenized_datasets['test'], #.select(range(10)),
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics
)

In [21]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.202100,0.163021,0.938760,0.938759


TrainOutput(global_step=1042, training_loss=0.244265284510812, metrics={'train_runtime': 1170.4571, 'train_samples_per_second': 21.359, 'train_steps_per_second': 0.89, 'total_flos': 6566466719729760.0, 'train_loss': 0.244265284510812, 'epoch': 1.0})

In [22]:
def classify_test(text):
  inputs = tokenizer(text, truncation=True, padding=True, return_tensors='pt' ).to("cuda")  # pytotch tensor: pt
  outputs = model(**inputs)

  probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
  predicted_class = torch.argmax(probabilities).item()
  return predicted_class

# print(classify_text(' It is somethiing to take the DSR bootcamp.'))
# print(classify_text('Today it is Tuesday some weeks passed from 34 years old.'))

In [23]:
trainer.evaluate()

{'eval_loss': 0.163021057844162,
 'eval_accuracy': 0.93876,
 'eval_f1': 0.9387589197073436,
 'eval_runtime': 295.6395,
 'eval_samples_per_second': 84.562,
 'eval_steps_per_second': 3.525,
 'epoch': 1.0}